In [6]:
from data_load import load_query_games, load_query_games, load_system_prompt
from utils import get_model_response, GPT_model
import pandas as pd
import os

In [13]:
def get_move_message(game, task_id, move_num):
    messages = []
    task = game.tasks[task_id]
    query_list = task['query_list']

    messages = []

    messages.append({"role": "system", "content": load_system_prompt('new_system_prompt.txt')})
    messages.append({"role": "user", "content": f"Name: {game.name}\nDescription: {game.description}"})
    messages.append({"role": "assistant", "content": game.code})

    
    messages.append({"role": "system", "content": load_system_prompt('query_prompt.txt')})
    for i in range(move_num):
        messages.append({"role": "user", "content": query_list[i].text})
        messages.append({"role": "assistant", "content": query_list[i].code})

    messages.append({"role": "user", "content": query_list[move_num].text})

    return messages



In [14]:
def get_few_shot_move_message(game, task_id, move_num, shots=2):
    messages = []
    task = game.tasks[task_id]
    query_list = task['query_list']

    messages = []

    messages.append({"role": "system", "content": load_system_prompt('system_prompt.txt')})
    messages.append({"role": "user", "content": f"Name: {game.name}\nDescription: {game.description}"})
    messages.append({"role": "assistant", "content": game.code})

    
    messages.append({"role": "system", "content": load_system_prompt('config_prompt.txt')})
    for i in range(min(shots,move_num)):
        messages.append({"role": "user", "content": query_list[i].text})
        messages.append({"role": "assistant", "content": query_list[i].code})

    messages.append({"role": "user", "content": query_list[move_num].text})

    return messages



In [15]:
def get_model_move_responses_df(game, task_id, message_fn=get_move_message):
    task = game.tasks[task_id]
    TOTAL_MOVES = len(task['query_list'])

    responses = []
    answers = []
    query = []
    for move_num in range(TOTAL_MOVES):
        model_response = get_model_response(message_fn(game, task_id, move_num))
        responses.append(model_response)
        answers.append(task['query_list'][move_num].code)
        query.append(task['query_list'][move_num].text)

    # make dataframe
    df = pd.DataFrame()
    df['responses'] = responses
    df['answers'] = answers
    df['query'] = query
    df['game_name'] = game.tag
    df['task_id'] = task_id

    return df
    

In [16]:
def get_game_responses_df(game, message_fn=get_move_message):
    TOTAL_TASKS = len(game.tasks)
    dfs = [get_model_move_responses_df(game, task_id) for task_id in range(TOTAL_TASKS)]
    df = pd.concat(dfs)
    return df

In [18]:
num_shots = 1
dir_path = f'{GPT_model}_predictions/{num_shots}_shots_moves/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
for game in load_query_games():
    print(f"Processing {game.tag}")
    message_fn = lambda game, task_id, move_num: get_few_shot_move_message(game, task_id, move_num, shots=num_shots)
    df = get_game_responses_df(game, message_fn=message_fn)
    df.to_csv(f'{dir_path}{game.tag}.csv', index=False)

Processing asimplegame
Processing tictactoe3d
Processing achi
Processing doguti


In [5]:
dir_path = f'{GPT_model}_predictions/moves/'
if not os.path.exists(dir_path):
    os.makedirs(dir_path)
for game in load_query_games():
    print(f"Processing {game.tag}")
    df = get_game_responses_df(game)
    df.to_csv(f'{dir_path}{game.tag}.csv', index=False)

Processing asimplegame
Processing tictactoe3d
Processing achi
Processing doguti
